In [1]:
# import module written for processing our gene mapping to gene ID, filter with e-value, pident
from network_analysis.map_string_to_ID import *

# node_path referes to diamond blastp --in REPRESENTING GENE --db string.dmnd
node_path = '/home/hermuba/nas2/resist_subnet/ecolinet/node_mapper.blast'
node = read_node_to_df(node_path)

# read the edge
edge_path = '/home/hermuba/data0118/map_to_exist_net/EcoliNet/EcoliNet.v1.txt'



In [2]:
# select nodes with high bitscore with string 511145 proteins
selected_nodes = filter_node(node)

/home/hermuba/resistanceExp/network_analysis/map_string_to_ID.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_nodes.drop_duplicates(subset = ['sseqid'], inplace = True)


In [3]:
selected_nodes.shape

(2614, 3)

In [4]:

edge = pd.read_csv(edge_path, sep = '\t', header = None, names = ['Protein A', 'Protein B', 'Score'])

In [5]:
edge.head()

,Protein A,Protein B,Score
0,b0723,b0724,6.255638
1,b0032,b0033,6.254619
2,b0722,b0723,6.254266
3,b0726,b0727,6.207392
4,b2751,b2752,6.204997


In [6]:
# map selected nodes to b number
root_dir = '/home/hermuba/data0118/map_to_exist_net/EcoliNet/'
b_number = pd.read_csv(root_dir+'ecoli_proteome.tab', sep = '\t')

In [7]:
b_number.head()

,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Gene names (ordered locus ),Gene names (ORF ),Gene names (primary ),Gene names (synonym )
0,P21515,ACPH_ECOLI,reviewed,Acyl carrier protein phosphodiesterase (ACP ph...,acpH yajB b0404 JW0394,Escherichia coli (strain K12),193,b0404 JW0394,NaN,acpH,yajB
1,P00509,AAT_ECOLI,reviewed,Aspartate aminotransferase (AspAT) (EC 2.6.1.1...,aspC b0928 JW0911,Escherichia coli (strain K12),396,b0928 JW0911,NaN,aspC,NaN
2,P0ABD5,ACCA_ECOLI,reviewed,Acetyl-coenzyme A carboxylase carboxyl transfe...,accA b0185 JW0180,Escherichia coli (strain K12),319,b0185 JW0180,NaN,accA,NaN
3,P26459,APPC_ECOLI,reviewed,Cytochrome bd-II ubiquinol oxidase subunit 1 (...,appC cbdA cyxA b0978 JW0960,Escherichia coli (strain K12),514,b0978 JW0960,NaN,appC,cbdA cyxA
4,P77398,ARNA_ECOLI,reviewed,Bifunctional polymyxin resistance protein ArnA...,arnA pmrI yfbG b2255 JW2249,Escherichia coli (strain K12),660,b2255 JW2249,NaN,arnA,pmrI yfbG


In [8]:
def get_bnumber(gene_names):
    
    if type(gene_names) == str:
        gene_names = gene_names.split(' ')
        for name in gene_names:
            if name[0] == 'b' and len(name) == 5:
                return name


b_number['EcoliNet_id'] = b_number['Gene names'].map(lambda x:get_bnumber(x))

In [9]:
selected_nodes.reset_index(inplace = True)

In [10]:
selected_nodes['sseqid'] = selected_nodes['sseqid'].str.split('|', expand = True)[2] # Entry Name
selected_nodes['EcoliNet-id'] = selected_nodes['sseqid'].map(b_number.set_index('Entry name')['EcoliNet_id'])




In [11]:
selected_nodes.dropna(inplace = True)
selected_nodes = selected_nodes.drop_duplicates('EcoliNet-id')

In [12]:
selected_nodes

,sseqid,qseqid,pident,evalue,EcoliNet-id
0,YAAI_ECOLI,AYIO01000001_12|1328432.3,99.3,3.600000e-70,b0013
2,RS20_ECOLI,AYIO01000001_27|1328432.3,100.0,1.100000e-40,b0023
4,SYI_ECOLI,AYIO01000001_29|1328432.3,99.7,0.000000e+00,b0026
6,FKBX_ECOLI,AYIO01000001_31|1328432.3,100.0,4.600000e-82,b0028
8,CAID_ECOLI,AYIO01000001_39|1328432.3,99.6,1.400000e-147,b0036
...,...,...,...,...,...
2606,YBFL_ECOLI,CYGI01000001_438|562.7784,97.7,2.900000e-174,b0705
2607,YHHI_ECOLI,CYGI01000001_438|562.7784,97.7,8.400000e-174,b3484
2609,YDCC_ECOLI,CYGI01000001_438|562.7784,97.0,2.700000e-172,b1460
2610,YBFD_ECOLI,CYGI01000001_438|562.7784,93.9,2.900000e-94,b0706


In [13]:
selected_nodes.set_index('EcoliNet-id', inplace = True)

In [14]:
selected_nodes['qseqid']

EcoliNet-id
b0013    AYIO01000001_12|1328432.3
b0023    AYIO01000001_27|1328432.3
b0026    AYIO01000001_29|1328432.3
b0028    AYIO01000001_31|1328432.3
b0036    AYIO01000001_39|1328432.3
                   ...            
b0705    CYGI01000001_438|562.7784
b3484    CYGI01000001_438|562.7784
b1460    CYGI01000001_438|562.7784
b0706    CYGI01000001_438|562.7784
b4503    CYGI01000001_438|562.7784
Name: qseqid, Length: 1242, dtype: object

In [15]:
edge['gene_one'] = edge['Protein A'].map(selected_nodes['qseqid'])
edge['gene_two'] = edge['Protein B'].map(selected_nodes['qseqid'])

edge.dropna(how = 'any', inplace = True) # we do not need interaction between genes that does not exist in our net

# drop useless columns
edge.drop(labels = ['Protein A', 'Protein B'], axis = 1, inplace = True)


In [16]:
edge.shape

(9224, 3)

In [17]:
edge.head()

,Score,gene_one,gene_two
21,6.078073,562.23305.con.0020_38|562.23305,562.58642.con.0075_7|562.58642
24,6.055138,AYIO01000009_414|1328432.3,AYIO01000009_415|1328432.3
29,6.033299,AYIO01000013_9|1328432.3,AYIO01000013_10|1328432.3
38,6.000272,AYIO01000025_77|1328432.3,AYIO01000025_78|1328432.3
44,5.973902,AYIO01000025_70|1328432.3,AYIO01000025_71|1328432.3


In [18]:
edge.to_csv('/home/hermuba/nas2/resist_subnet/ecolinet/ecolinet.mapped.csv')

In [19]:
edge

,Score,gene_one,gene_two
21,6.078073,562.23305.con.0020_38|562.23305,562.58642.con.0075_7|562.58642
24,6.055138,AYIO01000009_414|1328432.3,AYIO01000009_415|1328432.3
29,6.033299,AYIO01000013_9|1328432.3,AYIO01000013_10|1328432.3
38,6.000272,AYIO01000025_77|1328432.3,AYIO01000025_78|1328432.3
44,5.973902,AYIO01000025_70|1328432.3,AYIO01000025_71|1328432.3
...,...,...,...
95424,0.782926,562.65258.con.0004_105|562.65258,562.22664.con.0024_28|562.22664
95451,0.782577,AYIO01000009_125|1328432.3,FLWH01000020_32|562.12962
95492,0.781689,AYIO01000008_178|1328432.3,AYIO01000022_6|1328432.3
95506,0.781362,AYIO01000009_121|1328432.3,562.22449.con.0001_181|562.22449


In [20]:
import os
base_path = '/home/hermuba/nas2/resist_subnet/'
annotation_df = os.path.join(base_path, 'gold_anno.csv')
gold_anno = pd.read_csv(annotation_df, index_col = 0)
def PPV(computable, subset = None):
    if subset:
        subcomputable = computable.loc[(computable['target'].isin(subset))|(computable['source'].isin(subset))]
    else:
        subcomputable = computable.copy()
    grouped = subcomputable.groupby(by = ['goldstandard', 'prediction']).count()['target']
    
    try:
        tp = grouped[True, True]
        fp = grouped[False, True]
        PPV = tp/(tp+fp)
        
       
    except KeyError:
        PPV = 0
        
    return PPV

def coverage(computable, all_nodes):
    net = computable.loc[computable['prediction']]
    covered_nodes = set(net['source']).union(net['target']).intersection(all_nodes)
    
    return len(covered_nodes)/len(all_nodes)
    
def PPV_coverage(df, score_col, score_thres, gold_col = 'gosim', gold_thres = 0.6):
    ''' define goldstandard_score > gold_thres as Positive. d'''
    
    # write prediction
    computable = df.loc[(df[score_col].notnull())&(df[gold_col].notnull())]
    computable['prediction'] = df[score_col]>score_thres
    computable['goldstandard'] = df[gold_col]>gold_thres

    
    # total
    ppv = PPV(computable)
    cov = coverage(computable, set(gold_anno['gene_id']))
    
    # core
    core_genes = set(gold_anno.loc[gold_anno['core'], 'gene_id'])
    core_ppv = PPV(computable, subset = core_genes)
    core_cov = coverage(computable, core_genes)
    
    # acc
    acc_genes = set(gold_anno.loc[~gold_anno['core'], 'gene_id'])
    acc_ppv = PPV(computable, subset = acc_genes)
    acc_cov = coverage(computable, acc_genes)
    
    return ppv, core_ppv, acc_ppv, cov, core_cov, acc_cov

def try_thres(df, col, q = 300, interval = 10):
    tradeoff = []

    _, bins = pd.qcut(df[col], q = q, retbins = True, duplicates = 'drop')
    
    for bi in range(0, len(bins), interval):
        
        b = bins[bi]
        scores = PPV_coverage(df, col, b)
        tradeoff.append([b]+list(scores))
    return pd.DataFrame(tradeoff, columns = ['{}_score'.format(col), 'ppv', 'core_ppv', 'acc_ppv', 'cov', 'core_cov', 'acc_cov'])
    

In [21]:
# map edge to gene_id
edge['source'] = edge['gene_one'].map(gold_anno['gene_id'])
edge['target'] = edge['gene_two'].map(gold_anno['gene_id'])
edge['ecolinet'] = edge['Score']

In [22]:
# join gosim
gosim_df = pd.read_csv(os.path.join(base_path, 'gosim.csv'), sep = ' ', names = ['gene_id1', 'gene_id2', 'gosim'], skiprows = 1 )

In [23]:
import networkx as nx
gosim = nx.from_pandas_edgelist(gosim_df, source = 'gene_id1', target = 'gene_id2',  edge_attr='gosim')
ecolinet = nx.from_pandas_edgelist(edge, edge_attr='ecolinet')
combined = nx.compose(ecolinet, gosim)

In [24]:
edges_prediction = nx.to_pandas_edgelist(combined)

In [25]:
net='ecolinet'
d = try_thres(edges_prediction, net)
d.to_csv(os.path.join(base_path, 'preformance', '{}.ppv_cov.csv'.format(net)))

/home/hermuba/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/hermuba/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
core_genes = gold_anno.loc[gold_anno['core'] == True]
def net_stat(G):
    # core
    number_of_core = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['core'] == True]['gene_id']))
    # accessory
    number_of_acc = G.number_of_nodes() - number_of_core
    # hypothetical
    number_of_hypo = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['hypo_nr'] == True].index))
    # mobile
    number_of_mobile = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['is_aclame'] == True].index))
    # CARD genes
    number_of_card = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['is_card'] == True].index))
    # COG annotation
    number_of_cog = len(set(G.nodes()).intersection(gold_anno.loc[gold_anno['cog_ID'].notna()].index))
    return [G.number_of_nodes(), 
            G.number_of_edges(), 
            nx.algorithms.components.number_connected_components(G),
           number_of_core,
           number_of_acc,
           number_of_hypo,
           number_of_mobile,
           number_of_card,
           number_of_cog] 



In [27]:
net_stat(ecolinet)

[1095, 8854, 5, 964, 131, 0, 0, 0, 0]